**Instructions:**
This notebook can be run by first executing the code cells by pressing shift+enter while
selected or clicking the play button by the left of each. The GUI widgets then become
active and can be adjusted to modify the diagrams in real time.
Start with the 'Install/Import Packages' cell and proceed down sequentially. Files are saved to this cloud environment and can be accessed through the folder icon on the left.

---

## <center> Analytic 3D PIB Solutions <center/>

The particle in a box (PIB) is a quantum mechanical model system often used in introductory courses to teach concepts like normalization, boundary conditions, and energy quantization. In the 1-dimensional case of this system, a single particle is spatially confined due to an infinite potential of the form

$$
V(x) =
\begin{cases}
0  &\text{for} \ \ \ \ 0 < x < l_x \\
\infty &\text{for} \ \ \ \ l_x \leq x \leq 0
\end{cases}
$$

where $l_x $ is the length of the box.  Solving the time-independent Schrödinger equation, $\big( - \frac{\hbar^2}{2m}\frac{\textrm{d}^2}{{\textrm{d}x}^2} + V(x)\big) \psi_{n}(x) = E_{n}\psi_{n}(x)$, for the region inside the box yields the following analytic eigenfunction and eigenenergy solutions:

$$\psi_{n_x}(x) = \sqrt{\frac{2}{l_x}}\sin\Big(\frac{n_x\pi x}{l_x}\Big); \ \ \ \ E_{n_x} = \frac{{\pi}^2 \hbar^2 {n_x}^2}{2m{l_x}^2}$$

where $n_x$ is the quantum number. These simple, exact solutions can be extended to three dimensions through the
separation of variables method:

$$
\psi_{n_x,n_y,n_z}(x,y,z)=\sqrt{\frac{8}{l_xl_yl_z}}\
\sin\bigg(\frac{n_x\pi x}{l_x}\bigg)\sin\bigg(\frac{n_y\pi y}{l_y}\
\bigg)\sin\bigg(\frac{n_z\pi z}{l_z}\bigg).
$$

If the particle is taken to be an electron, the total energy can be expressed conveniently in Hartree atomic units ($\hbar=e=m_e=1$):

$$
E_{n_x,n_y,n_z}= \frac{\pi^2}{2}\
\bigg[\bigg(\frac{n_x}{l_x}\bigg)^2 + \bigg(\frac{n_y}{l_y}\bigg)^2 +\bigg(\frac{n_z}{l_z}\bigg)^2\bigg]
$$

Interactive visualizations of these 3D equations are realized in this notebook.

---

Try exploring the relationship between symmetry and degeneracy below by modifying the system parameters after executing the cells. As a point of comparison, here are the approximate box dimensions for the $\pi$ electrons in some polycyclic aromatic hydrocarbons considered in [Notebook 2](https://colab.research.google.com/github/tjz21/DFT_PIB_Code/blob/main/notebooks/NB2_PAH_HF.ipynb):

PAH|Rings|Dimensions (Bohr)|$\pi$ electrons|Fuse-type
:---:|:---:|:---:|:---:|:---:
benzene   |1|   8 x 8 x 3  | 6  |linear
naphthalene|2|  12 x 8 x 3 | 10 |linear
anthracene|3|  16 x 8 x 3 | 14 |linear
tetracene |4|  20 x 8 x 3 | 18 |linear
pentacene |5|  24 x 8 x 3 | 22 |linear
hexacene  |6|  28 x 8 x 3 | 26 |linear
heptacene |7|  32 x 8 x 3 | 30 |linear
---       |---|---       |--- |---
perylene  |5| 20 X 12 x 3 | 20 | non-linear
coronene  |7| 20 X 16 x 3 | 24 | non-linear

In [ ]:
#@title Install/Import Packages

# import standard anaconda packages
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=TypeError)
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Markdown, display, clear_output
import ipywidgets as widgets
from ipywidgets import Layout
from time import sleep
from termcolor import cprint
from os.path import exists
from google.colab import files
%config InlineBackend.figure_format = 'svg'

# setup 3rd party visualization library ipyvolume
!pip install -q ipyvolume==v0.6.3
import ipyvolume as ipv
import pythreejs
from google.colab import output
output.enable_custom_widget_manager()

def psi_reg(x, y, z, q_nx=1, q_ny=1, q_nz=1, lx=1, ly=1, lz=1):
  '''Numeric representation of the normalized 3D PIB wavefunction.
  Args:
    x, y, z (float or array): Cartesian spatial values.
    q_nx, q_ny, q_nz (int): Quantum numbers specifying state.
    lx, ly, lz (int): Box dimensions in Bohr.
  Returns:
    Wavefunction evaulated at a point or array if input x, y, z are arrays.
  '''
  wvfn = np.sqrt(8/(lx*ly*lz)) * \
  np.sin((q_nx*np.pi*x)/lx) * \
  np.sin((q_ny*np.pi*y)/ly) * \
  np.sin((q_nz*np.pi*z)/lz)
  return wvfn

def psi_ener(qnx, qny, qnz, lx, ly, lz):
  '''Calculates energy of 3D PIB state.
  Args:
    qnx, qny, qnz (int): Quantum numbers specifying state.
    lx, ly, lz (int): Box dimensions in Bohr.
  Returns:
    Float energy value in Ha.
  '''
  e_level = (4*np.pi**2/8)*((qnx/lx)**2 + (qny/ly)**2 + (qnz/lz)**2)
  return np.round(e_level, decimals=6)

def markdown_wvfn(q_nx, q_ny, q_nz, lx, ly, lz):
  '''Displays LaTeX equation of 3D wavefunction.
  Args:
    q_nx, q_ny, q_nz (int): Quantum numbers specifying state.
    lx, ly, lz (int): Box dimensions in Bohr.
  '''
  subscript = f'{q_nx},{q_ny},{q_nz}'
  sqrt_denom = f'({lx})({ly})({lz})'
  display(Markdown('## $$ \\text{State: \ \ } \psi_{' + subscript + '}(x,y,z)=' +
                           '\sqrt{\\frac{8}{' + sqrt_denom + '}}' +
                           '\sin{\\Big(\\frac{' + f'{q_nx}' '\pi x}{' + f'{lx}' + '}\\Big)}' +
                           '\sin{\\Big(\\frac{' + f'{q_ny}' '\pi y}{' + f'{ly}' + '}\\Big)}' +
                           '\sin{\\Big(\\frac{' + f'{q_nz}' '\pi z}{' + f'{lz}' + '}\\Big)} \\newline $$'))

def markdown_ener(l_x, l_y, l_z):
  '''Displays LaTeX equation of energy.
  Args:
    l_x, l_y, l_z (int): Box dimensions in Bohr.
  '''
  display(Markdown('## $$ E_{n_x,n_y,n_z} = \\frac{\pi^2}{2} \\Bigl[ ' +
                         ' \\Bigl(\\frac{n_x}{' + f'{l_x}' + '}\\Bigl)^2 +' +
                         ' \\Bigl(\\frac{n_y}{' + f'{l_y}' + '}\\Bigl)^2 +' +
                         ' \\Bigl(\\frac{n_z}{' + f'{l_z}' + '}\\Bigl)^2\\Bigl] \\newline$$'))

# Energy Diagram GUI widgets
num_elect_slider  = widgets.Dropdown(options=np.arange(2,27,2),value=6,description='electrons:',disabled=False)
lx_slider         = widgets.IntSlider(value=8,min=1,max=32,step=1,description='lx',disabled=False,readout_format='d',continuous_update=False)
ly_slider         = widgets.IntSlider(value=8,min=1,max=32,step=1,description='ly',disabled=False,readout_format='d',continuous_update=False)
lz_slider         = widgets.IntSlider(value=3,min=1,max=32,step=1,description='lz',disabled=False,readout_format='d',continuous_update=False)
length_labels     = widgets.Label(value='Box Lengths (Bohr): ')
box_length_ui     = widgets.HBox([length_labels, lx_slider, ly_slider, lz_slider],layout=widgets.Layout(border='solid 2px',width='%50'))
filename_text_mpl = widgets.Text(description='Filename (.png): ',value='PIB_ener',style={'description_width': 'initial'})
save_button_mpl   = widgets.Button(description='Save Image')

# Isosurface Rendering GUI widgets
lx_slider_iso     = widgets.IntSlider(value=8,min=1,max=32,step=1,description='lx',disabled=False,readout_format='d',continuous_update=False)
ly_slider_iso     = widgets.IntSlider(value=8,min=1,max=32,step=1,description='ly',disabled=False,readout_format='d',continuous_update=False)
lz_slider_iso     = widgets.IntSlider(value=3,min=1,max=32,step=1,description='lz',disabled=False,readout_format='d',continuous_update=False)
nx_slider         = widgets.IntSlider(value=1,min=1,max=10,step=1,description='nx',disabled=False,readout_format='d',continuous_update=False)
ny_slider         = widgets.IntSlider(value=1,min=1,max=10,step=1,description='ny',disabled=False,readout_format='d',continuous_update=False)
nz_slider         = widgets.IntSlider(value=1,min=1,max=10,step=1,description='nz',disabled=False,readout_format='d',continuous_update=False)
psi_square_check  = widgets.Checkbox(value=False, description=' ',disabled=False)
psi_check_ui      = widgets.HBox([widgets.Label(value='Probability density'), psi_square_check])
qnum_labels       = widgets.Label(value='Quantum Numbers: ')
qnum_ui           = widgets.HBox([qnum_labels, nx_slider, ny_slider, nz_slider])
box_length_iso_ui = widgets.HBox([length_labels, lx_slider_iso, ly_slider_iso, lz_slider_iso])
filename_text_ipv = widgets.Text(description='Filename (.png): ',value='PIB_iso',style={'description_width': 'initial'})
save_button_ipv   = widgets.Button(description='Save Image')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.7/260.7 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 271.7/271.7 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 15.9 MB/s eta 0:00:00


In [ ]:
#@title **Energy Diagram** (execute cell first)

def PIB_plotter(lx, ly, lz, num_elect, show=True, savefig=False, filename=None):
  '''Displays or saves a .png of 3D PIB energy level diagram.
  Args:
      lx, ly, lz (int): Box dimensions in Bohr.
      num_elect (int): Number of electrons.
      show (bool): Display the diagram.
      savefig (bool): Save diagram as .png file.
      filename (str): Filename to save to (without .png extension).
  '''
  ener_list = []
  # generate all possible combinations of quantum numbers for the 3D PIB sys
  # up to (nx=20, ny=20, nz=20) and find energies
  for i in range(1,20):
      for j in range(1,20):
          for k in range(1,20):
              ener_list.append(((i,j,k),psi_ener(qnx=i,qny=j,qnz=k, lx=lx, ly=ly, lz=lz)))
  # sort the list by energy values in ascending order
  ener_list.sort(key=lambda x: abs(x[1]))
  ener_list           = np.asarray(ener_list,dtype=object)
  ener_list[:,1]      = ener_list[:,1].astype(dtype=float)
  # find degeneracies, degen_list = (array([energy vals]), array([degeneracy of X]))
  degen_list          = np.unique(ener_list[:,1],return_counts=True)
  degen_log           = np.array([],dtype=int)
  for i in degen_list[1]:
      degen_log = np.append(degen_log, i*np.ones(i,dtype=int))
  degen_log           = degen_log.reshape(len(degen_log),1)
  ener_list           = np.hstack((ener_list,degen_log))
  ### find the unoccupied levels ###
  occ_levels          = int((num_elect/ 2))
  occ_states          = ener_list[0:occ_levels]
  occ_degen_accounted = np.where(occ_states[-1,1] == occ_states[:,1])[0].size
  occ_state_miss = occ_states[-1,2] - occ_degen_accounted
  if occ_state_miss > 0:
      occ_levels = occ_levels + occ_state_miss
      occ_states = ener_list[0:occ_levels]
  ### find the occupied levels ###
  unocc_levels          = 2
  unocc_states          = ener_list[occ_levels:occ_levels+unocc_levels]
  unocc_degen_accounted = np.where(unocc_states[-1,1] == unocc_states[:,1])[0].size
  unocc_state_miss      = unocc_states[-1,2] - unocc_degen_accounted
  if unocc_state_miss > 0:
      unocc_levels      = unocc_levels + unocc_state_miss
      unocc_states      = ener_list[occ_levels:occ_levels + unocc_levels]
  ################## Plotting commands ##################
  occ        = []
  unocc      = []
  energy_PIB = []
  for state in occ_states:
      occ.append(state[0])
      energy_PIB.append(round(state[1],8))
  for state in unocc_states:
      unocc.append(state[0])
      energy_PIB.append(round(state[1],8))
  PIBlevels  = occ + unocc
  # define x and y values for scatterplot
  yPIB       = np.array(energy_PIB)
  xPIB       = np.ones(yPIB.shape[0])
  for i in range(0,len(yPIB)):
      if yPIB[i] in yPIB[:i]:
          count = list(yPIB[:i]).count(yPIB[i])
          xPIB[i] += count*0.3 # offset for degenerate levels
  Gap_PIB    = round(energy_PIB[len(occ)] - energy_PIB[len(occ)-1], 3)
  Gap_PIB_ev = round(Gap_PIB*27.2114, 3)
  fig = plt.figure(figsize=(8, 5))
  ax = fig.add_subplot(1, 1, 1)
  plt.cla()
  plt.clf()
  clear_output(wait=True)
  plt.ylabel("Energy (Ha)",labelpad=7)
  plt.scatter(xPIB[len(occ):],yPIB[len(occ):],marker=0,s=1200,linewidths=6, color='#F97306', label='virtual')
  plt.scatter(xPIB[:len(occ)],yPIB[:len(occ)],marker=0,s=1200,linewidths=6, color='green', label='occupied')
  plt.rcParams["legend.markerscale"] = 0.45
  plt.legend(loc='upper right', handlelength=3,handletextpad=.1)
  plt.xticks([])
  plt.xlim([0.3,3])
  plt.ylim([min(yPIB)-0.1,max(yPIB)+0.1])
  annotations = [str(x) for x in PIBlevels]
  for i, label in enumerate(annotations):
        if list(yPIB).count(yPIB[i])==1:
            plt.annotate(label, (xPIB[i] + 0.005, yPIB[i]),size=8)
            plt.text(xPIB[i]-0.46, yPIB[i], "{:.3f}".format(yPIB[i]),size=8)
        else:
            plt.annotate(label, (xPIB[i] - 0.25, yPIB[i] - 0.015),size=8)
            if yPIB[i] not in yPIB[:i]:
                plt.text(xPIB[i]-0.46, yPIB[i], "{:.3f}".format(yPIB[i]),size=8)
  plt.title('3D PIB Energy Diagram')
  plt.text(0.93, 0.88, 'States: (n$_x$, n$_y$, n$_z$)',
           horizontalalignment='center', verticalalignment='center',
           transform=ax.transAxes,weight='bold')
  plt.text(0.93, 0.810,f'H-L Gap: {Gap_PIB} Ha\n              ({Gap_PIB_ev} ev)',
           size=10.5, horizontalalignment='center', verticalalignment='center',
           transform= ax.transAxes)
  ax.spines['left'].set_position(('axes', 0.16))
  ax.spines['top'].set_visible(False)
  ax.spines['right'].set_visible(False)
  ax.spines['bottom'].set_visible(False)
  plt.tight_layout()
  if savefig == True:
    plt.savefig(f'{filename}.png', dpi=800)
  if show == True:
    plt.show()
  else:
    plt.close()

def on_click_save(b):
  '''Saves a .png file of diagram with widget parameters.
  Args:
      b : button click
  '''
  PIB_plotter(lx_slider.value, ly_slider.value, lz_slider.value,
              num_elect_slider.value, show=False, savefig=True,
              filename=filename_text_mpl.value)

save_button_mpl.on_click(on_click_save) # link save button to save function

# attach sliders to diagram
ener_plot_out = widgets.interactive_output(PIB_plotter, {'lx':lx_slider,
                                                        'ly':ly_slider,
                                                        'lz':lz_slider,
                                                        'num_elect':num_elect_slider})
# attach sliders to LaTeX equation
energy_display = widgets.interactive_output(markdown_ener, {'l_x':lx_slider,
                                                            'l_y':ly_slider,
                                                            'l_z':lz_slider})
# generate the interface
display(num_elect_slider,
        box_length_ui,
        energy_display,
        Markdown('<br>'),
        ener_plot_out,
        widgets.HBox([filename_text_mpl,save_button_mpl]));

Dropdown(description='electrons:', index=2, options=(2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26), value=6)

Output()

<br>

Output()

In [ ]:
#@title **Isosurface Rendering**

def isoplotter(nx_val, ny_val, nz_val, lx, ly, lz, psi_square=False, plot_save=True):
  '''Displays and saves isosurface of 3D PIB wavefunction.
  Args:
      nx_val, ny_val, nz_val (int): Quantum numbers specifying state.
      lx, ly, lz (int): Box dimensions in Bohr.
      psi_square (bool): Calculate prob. density (true) or wavefunction (false).
      plot_save (bool): Save a .png file of display.
  '''
  # construct 3d grid of points
  nx_p, ny_p, nz_p = 7 * lx, 7 * ly, 7 * lz
  xp, yp, zp       = np.linspace(0, lx, nx_p), np.linspace(0, ly, ny_p), np.linspace(0, lz, nz_p)
  X, Y, Z          = np.meshgrid(xp, yp, zp, indexing='ij')
  psi              = psi_reg(X, Y, Z, nx_val, ny_val, nz_val, lx, ly, lz)
  norm_psi         = psi_reg(X, Y, Z, nx_val, ny_val, nz_val, lx, ly, lz)**2

 # ipyvolume potting commands
  ipv.clear()
  fig = ipv.figure(title='PIB',width=500, height=500)
  fig.camera.type = 'OrthographicCamera'
  if psi_square:
    norm_sur = ipv.pylab.plot_isosurface(norm_psi, color='red', level=norm_psi.mean(), controls=True,
                                         description='prob. density')
  else:
      pos_values = np.ma.array(psi, mask = psi < 0.0)
      if nx_val == ny_val == nz_val == 1:
        pos_sur = ipv.pylab.plot_isosurface(psi,color='red', level=np.sqrt(norm_psi.mean()), controls=True,
                                            description='positive')
      else:
        pos_sur = ipv.pylab.plot_isosurface(psi, color='red', level=np.sqrt(norm_psi.mean()), controls=True,
                                            description='positive')
        neg_sur = ipv.pylab.plot_isosurface(psi, color='blue', level=-np.sqrt(norm_psi.mean()), controls=True,
                                            description='negative')
  ipv.style.box_off()
  ipv.squarelim()
  ipv.view(0,-75)
  ipv.xyzlabel('lx', 'ly', 'lz')
  ipv.show()

def plot_saver(b):
  '''Saves a .png file of display. '''
  ipv.savefig(f'{filename_text_ipv.value}.png', width=1200, height=1200)

save_button_ipv.on_click(plot_saver) # link button and save action

# link sliders to isosurface function
out = widgets.interactive_output(isoplotter, {'nx_val':nx_slider,
                                              'ny_val':ny_slider,
                                              'nz_val':nz_slider,
                                              'lx':lx_slider_iso,
                                              'ly':ly_slider_iso,
                                              'lz':lz_slider_iso,
                                              'psi_square':psi_square_check})
# link sliders to LaTeX equation
wavefunction_display = widgets.interactive_output(markdown_wvfn, {'q_nx':nx_slider,
                                                                  'q_ny':ny_slider,
                                                                  'q_nz':nz_slider,
                                                                  'lx':lx_slider_iso,
                                                                  'ly':ly_slider_iso,
                                                                  'lz':lz_slider_iso})
# generate the interface
display(out,
        wavefunction_display,
        qnum_ui,
        box_length_iso_ui,
        psi_check_ui,
        Markdown('<sub><sup>(defined as $\\rho(x,y,z) = |\psi(x,y,z)|^2 $)<sub/><sup/>'),
        widgets.HBox([filename_text_ipv,save_button_ipv]))

Output()

Output()

<sub><sup>(defined as $\rho(x,y,z) = |\psi(x,y,z)|^2 $)<sub/><sup/>


## **References**

* Atkins, P. W.; de Paula, J. *Physical Chemistry*, 9th ed.; W. H. Freeman: New York, NY, **2007**.

* Silbey, R. J.; Alberty, R. A.; Bawendi, M. G. *Physical Chemistry*, 4th ed.; John Wiley & Sons, Inc: Hoboken, NJ, **2005**.
